# Prelude



[**Resonators**](https://github.com/0xmaddie/0xmaddie/blob/main/resonators)
is a Python library that uses data races and particle swarm dynamics
to approximate functions. You can train a resonator like a transformer
but it uses Python objects instead of vectors.

You can do program evolution with a stack of "genetic transformer" blocks each containing fitness, crossover, and mutation operations. attach weighted coins to race all of the operations and train this with a particle swarm on data showing the evolutionary paths you want.

You can increase representational capacity by expressing conditional dependence between the weights. so per block there are F fitness weights, FxC crossover weights, FxCxM mutation weights representing the probaility that operation will be next in line during its race, along with 1 weight for a type of residual connection.

The crossover operations combine several states in to one and the mutation operations transform one state in to several. This acts as a type of information bottleneck and provides an analogy to continuity in that it "smooths out" the details between states.


# Setup

In [ ]:
import google
google.colab.drive.mount('/content/drive')
!cp /content/drive/MyDrive/secret.py .
import secret

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
!pip install git+https://github.com/0xmaddie/0xmaddie.git#subdirectory=lisp
!pip install git+https://github.com/0xmaddie/0xmaddie.git#subdirectory=llama
!pip install git+https://github.com/0xmaddie/0xmaddie.git#subdirectory=resonators

In [ ]:
import lisp
import llama
import resonators
import bs4

In [ ]:
!python -m unittest lisp --verbose

In [ ]:
#_model     = llm.model
#_tokenizer = llm.tokenizer
llm = llama.Llama(
  system_prompt=f'''
You are a helpful assistant.
'''.strip(),
  temperature=0.7,
  max_new_tokens=1024,
  purity=1,
  quota=3,
  #model=_model,
  #tokenizer=_tokenizer,
)

# A text-to-image application

In [ ]:
def get_tag_body(document, tag_name):
  # Perform string replacement to transform <value> tags into <div class="value">
  document = document.replace(f"<{tag_name}>", f'<div class="{tag_name}">').replace(f"</{tag_name}>", "</div>")
  # Parse the modified HTML document with BeautifulSoup
  soup = bs4.BeautifulSoup(document, 'html.parser')
  # Find all divs with the class "value"
  value_divs = soup.find_all('div', class_=tag_name)
  # Ensure there is exactly one div with class "value"
  if len(value_divs) != 1:
    raise ValueError(f"Expected exactly one <div class='{tag_name}'> tag, but found {len(value_divs)}.")
  # Return the text content of the <div class="value">
  return value_divs[0].get_text()

class Repository:
  llm: llama.Llama

  def __init__(self, llm):
    self.llm = llm

  def get_fitness(self, operation):
    def map_state(state):
      value = get_tag_body(state, 'value')
      match value:
        case 'True':
          return 1
        case 'False':
          return 0
        case _:
          raise ValueError(f'Expected a bool, but got {state}')
      return value
    def reduce_states(states):
      average = sum(states)/len(states)
      return average > 0.5
    def hidden(state):
      prompt = f'''
Consider the following generative text-to-image prompt:

<prompt>{state}</prompt>

{operation} Think carefully, then respond with either True or False
within <value></value> tags.
'''.strip()
      target = self.llm.reduce([prompt], map_state, reduce_states)
      return target
    return hidden

  def get_crossover(self, operation):
    def map_state(state):
      prompt = get_tag_body(state, 'prompt')
      return prompt
    def reduce_states(states):
      return states[0]
    def hidden(fst, snd):
      prompt = f'''
Consider these two generative text-to-image prompts:

<prompt>{fst}</prompt>
<prompt>{snd}</prompt>

{operation} Think carefully, then respond with the final
prompt within <prompt></prompt> tags.
'''.strip()
      target = self.llm.reduce([prompt], map_state, reduce_states)
      return target
    return hidden

  def get_mutation(self, operation):
    def map_state(state):
      prompt = get_tag_body(state, 'prompt')
      return prompt
    def reduce_states(states):
      return states[0]
    def hidden(state):
      prompt = f'''
Consider the following generative text-to-image prompt:

<prompt>{state}</prompt>

{operation} Think carefully, then respond with the final
prompt within <prompt></prompt> tags.
'''.strip()
      target = self.llm.reduce([prompt], map_state, reduce_states)
      return target
    return hidden

In [ ]:
repo = Repository(llm)

fitness = [
  repo.fitness('Does this prompt mention anything related to anime?'),
  repo.fitness('Does this prompt mention any famous artists?'),
  repo.fitness('Does this prompt mention a famous location?'),
  repo.fitness('Does this prompt mention flowers?'),
  repo.fitness('Does this prompt mention animals?'),
  repo.fitness('Would this prompt be popular on Reddit?'),
  repo.fitness('Do you think images generated by this prompt would have people in them?'),
  repo.fitness('Do you think images generated by this prompt would have animals in them?'),
  repo.fitness('Does this prompt mention any specific historical art styles?'),
  repo.fitness('Does this prompt include a famous artist from before 1912?'),
  repo.fitness('Does this prompt involve a modern artist or art movement?'),
  repo.fitness('Does this prompt specify an artistic medium, like oil painting or sculpture?'),
  repo.fitness('Does this prompt mention a specific time period for the art style?'),
  repo.fitness('Does this prompt refer to a well-known historical event?'),
  repo.fitness('Does this prompt mention a specific location known for its artistic heritage?'),
  repo.fitness('Does this prompt involve mythological figures or stories?'),
  repo.fitness('Does this prompt feature any fictional characters?'),
  repo.fitness('Does this prompt mention specific colors that should dominate the artwork?'),
  repo.fitness('Does this prompt include specific elements of nature like mountains or rivers?'),
  repo.fitness('Does this prompt mention urban settings or cityscapes?'),
  repo.fitness('Does this prompt feature musical elements or musicians?'),
  repo.fitness('Does this prompt include elements that are surreal or abstract?'),
  repo.fitness('Does this prompt reference specific cultural symbols or national heritage?'),
  repo.fitness('Is the prompt likely to generate interest among art historians?'),
  repo.fitness('Does this prompt mention elements that would appeal to children?'),
  repo.fitness('Does this prompt specify a mood or atmosphere, like melancholy or joy?'),
  repo.fitness('Does this prompt suggest a narrative or scene with action?'),
  repo.fitness('Does this prompt include technological elements or futuristic themes?'),
  repo.fitness('Does this prompt have potential educational value?'),
  repo.fitness('Would images generated from this prompt likely be controversial?'),
  repo.fitness('Is the imagery likely to be peaceful or calming?'),
  repo.fitness('Would this prompt be considered unique or rare?'),
  repo.fitness('Does this prompt involve seasonal themes like winter or autumn?'),
  repo.fitness('Does this prompt involve water elements like oceans or lakes?'),
  repo.fitness('Does this prompt encourage exploring dark or horror themes?'),
  repo.fitness('Does this prompt involve celestial elements like stars or planets?'),
  repo.fitness('Is this prompt likely to engage amateur artists?'),
  repo.fitness('Does this prompt involve famous fictional stories or books?')
]

crossover = [
  repo.crossover('Combine these prompts in the most natural way possible.'),
  repo.crossover('Combine these prompts in a surprising way.'),
  repo.crossover('Think of another prompt that is a logical consequence of these prompts.'),
  repo.crossover('Combine these prompts to create a harmonious blend of their themes.'),
  repo.crossover('Merge these prompts while emphasizing any historical elements present in both.'),
  repo.crossover('Create a new prompt that juxtaposes the primary subjects of these prompts in an unexpected way.'),
  repo.crossover('Integrate the artistic styles mentioned in these prompts to form a new hybrid style.'),
  repo.crossover('Combine these prompts in a way that tells a story or creates a narrative.'),
  repo.crossover('Fuse these prompts while focusing on a dominant element from each.'),
  repo.crossover('Combine these prompts to highlight contrast, such as modern versus ancient or natural versus man-made.'),
  repo.crossover('Synthesize these prompts into a single prompt that would appeal to a specific audience or age group.'),
  repo.crossover('Create a crossover that results in an artwork focusing on the interaction of elements from each prompt.'),
  repo.crossover('Develop a prompt that would result in an artwork featuring a clash of the cultures or themes mentioned.'),
  repo.crossover('Construct a new prompt that would feature elements of fantasy or surrealism from the original prompts.'),
  repo.crossover('Merge these prompts to enhance the visual complexity of the resulting image.'),
  repo.crossover('Combine these prompts to emphasize environmental or natural elements present in both.'),
  repo.crossover('Formulate a new prompt that integrates the emotional or atmospheric tones of both prompts.'),
  repo.crossover('Design a crossover that results in an educational or informative artwork combining elements of both prompts.'),
  repo.crossover('Create a prompt that combines technological and historical themes from the original prompts.'),
  repo.crossover('Construct a prompt that merges the specific locations mentioned in the original prompts into a single scene.'),
  repo.crossover('Combine these prompts to create an artwork that could serve as a metaphor or allegory.'),
  repo.crossover('Synthesize these prompts in a way that would result in an abstract or minimalistic artwork.'),
  repo.crossover('Develop a prompt that balances elements of action and calm from the original prompts.')
]

mutation = [
  repo.mutation('Make the theme of this prompt anime.'),
  repo.mutation('Shorten this prompt to the absolute bare minimum while keeping the same meaning.'),
  repo.mutation('Make the theme of this prompt ancient Greece.'),
  repo.mutation('Make the theme of this prompt ancient Rome.'),
  repo.mutation('Reinterpret this prompt as an oil painting.'),
  repo.mutation('Reinterpret this prompt as a watercolor painting.'),
  repo.mutation('Reinterpret this prompt as experimental photography.'),
  repo.mutation('Transform this prompt into a futuristic or sci-fi theme.'),
  repo.mutation('Convert this prompt into a children’s book illustration style.'),
  repo.mutation('Adapt this prompt to the style of a famous artist not previously mentioned.'),
  repo.mutation('Modify this prompt to include surreal or abstract elements.'),
  repo.mutation('Change this prompt to focus on a specific season like winter or summer.'),
  repo.mutation('Expand this prompt to include a detailed background setting.'),
  repo.mutation('Alter this prompt to include a specific animal or creature.'),
  repo.mutation('Make this prompt evoke a specific emotion like joy or sadness.'),
  repo.mutation('Add a mythical or legendary component to this prompt.'),
  repo.mutation('Shift this prompt’s setting from day to night.'),
  repo.mutation('Enhance this prompt with a focus on dynamic movement or action.'),
  repo.mutation('Incorporate a moral or philosophical question into this prompt.'),
  repo.mutation('Adjust this prompt to emphasize architectural elements.')
]

In [ ]:
initial_states = [
  "A digital painting of a bustling cityscape in the Renaissance era, bustling with traders and artists.",
  "A charcoal drawing of a serene garden filled with various types of flowers and butterflies.",
  "A surrealistic scene featuring mythical creatures interacting in a futuristic urban setting.",
  "An abstract oil painting depicting the emotional intensity of a violin player in a dimly lit room.",
  "A watercolor painting of a quiet village by the sea during the golden hour, reflecting soft, warm lights.",
  "A graphite sketch of famous landmarks from ancient Rome, including the Colosseum and Roman Forum.",
  "A series of experimental photographs capturing the bustling nightlife of Tokyo.",
  "A large-scale mural design incorporating elements of Native American folklore and landscape.",
  "An ink drawing of a medieval battle scene, illustrating detailed armor and weaponry of the era.",
  "A digital collage representing a fusion of various modern art styles seen in New York’s contemporary galleries.",
  "A minimalist pastel drawing of a snowy landscape with subtle hints of wildlife presence.",
  "A vibrant anime-style poster featuring a heroic character poised in a dynamic action pose.",
  "A classical portrait of a famous historical figure from the Enlightenment period in a detailed study room.",
  "An experimental mixed media piece that combines classical Greek sculpture with modern abstract forms.",
  "A Victorian-era street scene depicted in the style of Impressionism, focusing on light and shadow effects."
]
initial_states = numpy.random.permutation(initial_state)
layers = 1
num_weights = resonators.Decoder.num_weights_for_components(
  fitness, crossover, mutation, layers,
)
decoder = resonators.Decoder(
  fitness=fitness,
  crossover=crossover,
  mutation=mutation,
  weights=numpy.random.random(num_weights),
  state_capacity=len(initial_states),
  state_bottleneck=2,
)
final_states = decoder(initial_states)
print(final_states)

# Junk

In [ ]:
import numpy as np
import scipy

class Decoder:
  def __init__(self, inputs, outputs, weights, purity=100, width=12, threshold=4):
    self.inputs    = inputs
    self.outputs   = outputs
    self.weights   = weights
    self.purity    = purity
    self.width     = width
    self.threshold = threshold

  @property
  def density(self):
    return len(self.inputs)*len(self.outputs)

  # The number of weights per layer.
  @property
  def layer_capacity(self):
    return len(self.inputs) + len(self.inputs) * len(self.outputs) + 1

  @property
  def layers(self):
    return len(self.weights)//self.layer_capacity

  @property
  def capacity(self):
    return len(self.weights)

  # The weights associated with the inputs for a certain layer.
  def win(self, layer):
    lhs = layer * self.capacity
    return self.weights[lhs:lhs+len(self.inputs)]

  # The weights associated with the outputs for a certain layer.
  def wout(self, layer, input_index):
    input_lhs  = layer * self.capacity
    output_lhs = input_lhs + len(self.inputs)
    target_lhs = output_lhs + input_index * len(self.outputs)
    return self.weights[target_lhs:target_lhs+len(self.outputs)]

  # The weights associated with the residual for a certain layer.
  def wres(self, layer):
    input_lhs    = layer * self.capacity
    residual_idx = input_lhs + len(self.inputs) + len(self.inputs) * len(self.outputs)
    return self.weights[residual_idx]

  def sort(self, components, weights):
    # Each weight is the probability that the corresponding
    # component will be next in line.
    indices = np.random.choice(
      len(self.inputs),
      size=len(self.inputs),
      replace=False,
      p=scipy.special.softmax(weights),
    )
    return indices

  def __call__(self, swarm):
    for time in range(self.layers):
      inputw    = self.win(time)
      residualw = self.wres(time)
      hidden    = []
      for i in range(self.purity):
        print(f'Decoder.__call__ iteration # {i}')
        # The inputs race to provide a measurement.
        sample = []
        inputbest = None
        inputlist = self.sort(self.inputs, inputw)
        inputlist = list(reversed(inputlist))
        while inputbest is None and len(inputlist) > 0:
          inputid = inputlist.pop()
          input = self.inputs[inputid]
          sample.clear()
          for state in swarm:
            if input(state):
              print(f'Decoder.__call__ inputid={inputid} {state} True')
              sample.append(state)
            else:
              print(f'Decoder.__call__ inputid={inputid} {state} False')
            if len(sample) >= self.threshold:
              inputbest = inputid
              break
        # If no inputs were able to provide a large enough measurement
        # then an exception is raised.
        if inputbest is None:
          print(f'Decoder.__call__ inputid={inputid} Err')
          raise ValueError()
        assert len(sample) == self.threshold
        print(f'Decoder.__call__ inputbest={inputbest} Ok')
        activity = False
        # The outputs condition on the winner of the race
        # and perform their own race to provide a state.
        outputw = self.wout(time, inputbest)
        print(f'\n\n------\n\nDecoder.__call__ OUTPUT\n\n-----\n\n')
        for outputid in self.sort(self.outputs, outputw):
          output = self.outputs[outputid]
          try:
            # The output is applied to *all* of the sample values.
            # It's important to mix states across the swarm to allow
            # the analogy to continuity.
            point    = output(sample)
            activity = True
            hidden.append(point)
            print(f'Decoder.__call__ outputid={outputid} {point} Ok')
            break
          except ValueError:
            print(f'Decoder.__call__ outputid={outputid} False')
        if not activity:
          print(f'Decoder.__call__ outputid={outputid} Err')
          raise ValueError()
      # The final states race with the initial states.
      target = []
      for i in range(self.purity):
        if np.random.random() < residualw:
          state = np.random.choice(hidden)
          print(f'Decoder.__call__ residual hidden {state}')
        else:
          state = np.random.choice(swarm)
          print(f'Decoder.__call__ residual initial {state}')
        target.append(state)
      swarm = target
    return swarm

In [ ]:
class Repository:
  llm: Llama

  def __init__(self, llm):
    self.llm = llm

  def input(self, input):
    def map_state(x):
      value = get_tag_body(x, 'value')
      print(f'input/map_state: value={value}')
      match value:
        case 'True':
          return True
        case 'False':
          return False
        case _:
          raise ValueError(f'Repository.input unknown value {value}')
    def reduce_state(xs):
      numbers = [1 if bool(x) else 0 for x in xs]
      average = sum(numbers)/len(xs)
      print(f'input/reduce_state: xs={xs} average={average}')
      return average > 0.5
    def body(state):
      get_state = f'''
Consider the following text-to-image prompt:

```
{state}
```

{input}

Think about it carefully, then respond with either True or False
in <value> tags, like this: <value>True</value> or <value>False</value>.
'''.strip()
      state = self.llm.reduce([get_state], map_state, reduce_state)
      print(f'input/state={state}')
      return state
    return body

  def output(self, output):
    def map_output(x):
      value = get_tag_body(x, 'value')
      print(f'output/map_output: value={value}')
      return value
    def reduce_output(xs):
      print(f'output/reduce_output: xs={xs}')
      return xs[0]
    def body(input, sample):
      get_state = f'''
Consider the following text-to-image prompts:

```
{sample}
```

These prompts have something in common: they all satisfy the
following condition:

```
{input}
```

and they are going to be transformed according to the following
operation:

```
{output}
```

Think of an interesting way to combine all of these prompts in
to a single prompt based on this information. When you come up
with a prompt, put it in <value></value> tags. Keep your prompt
short and terse while including as much visual detail as
possible.
'''.strip()
      def map_state(x):
        value = get_tag_body(x, 'value')
        print(f'output/map_state: x={x}')
        return value
      def reduce_state(xs):
        print(f'output/map_state: xs={xs}')
        return xs[0]
      state = self.llm.reduce([get_state], map_state, reduce_state)
      print(f'output/state = {state}')
      return state
    return body

In [ ]:
repo = Repository(llm)

inputs = [
    # repo.input('Does this prompt mention men other than the artist?'),
    # repo.input('Does this prompt mention women other than the artist?'),
    repo.input('Does this prompt mention a famous artist?'),
    repo.input('Does this prompt mention people other than the artist?'),
    repo.input('Does this prompt not mention people other than the artist?'),
    # repo.input('Is this a classical painting?'),
    repo.input('Is this an anime illustration?'),
    # repo.input('Does this artwork belong to the Renaissance period?'),
    repo.input('Is this a modern abstract artwork?'),
    repo.input('Is this piece in the style of Impressionism?'),
    # repo.input('Does this resemble a Baroque painting?'),
    # repo.input('Is this artwork in the style of Cubism?'),
    # repo.input('Is the artist associated with the Surrealism movement?'),
    repo.input('Does this image use techniques typical of watercolor painting?'),
    repo.input('Is this artwork a digital creation?'),
    repo.input('Does this prompt mention Pixiv?'),
    repo.input('Does this prompt mention a man?'),
    repo.input('Does this prompt mention a woman?'),
    repo.input('Does this prompt mention "illustration"?'),
    # repo.input('Does this represent a form of street art?'),
    # repo.input('Is this a piece of Gothic art?'),
    # repo.input('Is this image in the style of Art Nouveau?'),
    # repo.input('Is this artwork part of the Pop Art movement?'),
    repo.input('Does this belong to the Abstract Expressionism style?'),
    # repo.input('Is this a Realism artwork?'),
    # repo.input('Does this represent Minimalism?'),
    # repo.input('Is this artwork done in the Art Deco style?'),
    # repo.input('Does this piece use techniques of Neo-Impressionism?'),
    # repo.input('Is this artwork influenced by Futurism?'),
]
outputs = [
    repo.output('Make this an anime illustration.'),
    repo.output('Make this a classical painting.'),
    repo.output('Transform this image to reflect the Renaissance art style.'),
    repo.output('Create this image in a modern abstract style.'),
    repo.output('Convert this to an Impressionist style painting.'),
    repo.output('Reimagine this as a Baroque period painting.'),
    repo.output('Adapt this image to the Cubism art style.'),
    repo.output('Render this image in a Surrealist style.'),
    repo.output('Use watercolor techniques for this image.'),
    repo.output('Create this artwork using digital media.'),
    repo.output('Style this image as street art.'),
    repo.output('Transform this into a Gothic art piece.'),
    repo.output('Create this image in the style of Art Nouveau.'),
    repo.output('Convert this to the Pop Art style.'),
    repo.output('Adapt this image to Abstract Expressionism.'),
    repo.output('Render this image as a Realism artwork.'),
    repo.output('Style this artwork with Minimalist techniques.'),
    repo.output('Create this in the Art Deco style.'),
    repo.output('Use Neo-Impressionist techniques for this image.'),
    repo.output('Reimagine this artwork in the style of Futurism.'),
]

In [ ]:
is_classical = inputs[0]
fst = 'A digital art illustration by Ilya Kuvshinov.'
snd = 'An oil painting by John Singer Sargent'
print(f'{fst} = {is_classical(fst)}')
print(f'{snd} = {is_classical(snd)}')

make_anime = outputs[0]
response = make_anime(
  'Is this a classical painting?',
  [
    'An oil painting of a woman holding a parasol by John Singer Sargent',
    'A still life of fruits on a table by Joaquin Sorolla',
    'A scene from Greek mythology by Leonardo da Vinci',
  ],
)